<a href="https://colab.research.google.com/github/Zhikharevi/CAOSProgect/blob/main/Python_2_HW_4_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Продвинутый Python, ДЗ-4

Правила игры:

В домашке 6 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью plotly или же Dash. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=81bcb65231849967113bb68e4361dfc487a622fc28b2549e07fdc85ebe677b3e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
locations = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

# Часть 1. Plotly

В данной части для заданий нужно использовать Plotly для того, чтобы отобразить графики

In [4]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

## Задание 1 (15 баллов)

Отобразите на карте всех продавцов (в качестве точек - координаты, в качестве размера и цвета точек - число продавцов). При наведении точек должно отображаться название штата, города, а также число продавцов

*Hint: внимательно посмотрите на датасет locations. Если делать задачу максимально втупую, то у вас либо крашнется/зависнет отрисовка, либо на это нельзя будет нормально смотреть*

In [5]:
filtered_locations = locations.drop(['geolocation_city', 'geolocation_state'], axis = 1)

aggregated_locations = filtered_locations.groupby('geolocation_zip_code_prefix')\
                                         .agg(lat = ('geolocation_lat', 'mean'),
                                              lng = ('geolocation_lng', 'mean'))\
                                         .reset_index()

sellers_to_locations = pd.merge(sellers, aggregated_locations, \
                                left_on = 'seller_zip_code_prefix', right_on = 'geolocation_zip_code_prefix')\
                         .drop(['geolocation_zip_code_prefix'], axis = 1)

result = sellers_to_locations.groupby(['seller_zip_code_prefix', 'seller_city', 'seller_state', 'lat', 'lng'])\
                             .agg(seller_count = ('seller_id','count')).reset_index()\
                             .drop(['seller_zip_code_prefix'], axis = 1)

In [6]:
result.head()

,seller_city,seller_state,lat,lng,seller_count
0,sao paulo,SP,-23.550190,-46.634024,1
1,sao paulo,SP,-23.543000,-46.632143,2
2,sao paulo,SP,-23.544922,-46.631931,1
3,sao paulo,SP,-23.542331,-46.631373,2
4,sao paulo,SP,-23.539657,-46.631884,8


In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2289 entries, 0 to 2288
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   seller_city   2289 non-null   object 
 1   seller_state  2289 non-null   object 
 2   lat           2289 non-null   float64
 3   lng           2289 non-null   float64
 4   seller_count  2289 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 89.5+ KB


In [8]:
fig = px.scatter_geo(result, lat='lat', lon='lng',
                     color="seller_count",
                     size='seller_count',
                     hover_name="seller_city",
                     hover_data=["seller_state", "seller_count"],
                     projection="natural earth")

fig.show()

## Задание 2 (15 баллов)

Отобразите на графике число заказов по дням с разбивкой по статусам. Также создайте график со средней оценкой отзыва (аггрегируйте по дням). Для обоих графиков должен быть слайдер по датам

In [9]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp']).apply(lambda x: x.date())
order_counts = orders.groupby(['order_purchase_timestamp', 'order_status'])\
                     .agg(order_count = ('order_id','count')).reset_index()
order_counts

,order_purchase_timestamp,order_status,order_count
0,2016-09-04,shipped,1
1,2016-09-05,canceled,1
2,2016-09-13,canceled,1
3,2016-09-15,delivered,1
4,2016-10-02,canceled,1
...,...,...,...
2150,2018-09-29,canceled,1
2151,2018-10-01,canceled,1
2152,2018-10-03,canceled,1
2153,2018-10-16,canceled,1


In [10]:
fig1 = px.line(order_counts, x = 'order_purchase_timestamp', y = 'order_count', color = 'order_status')
fig1.update_xaxes(title = 'дата', rangeslider_visible = True)
fig1.update_yaxes(title = 'число заказов')
fig1.show()

In [11]:
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date']).apply(lambda x: x.date())

review_scores = reviews.groupby('review_creation_date').agg(avg_score = ('review_score','mean')).reset_index()
review_scores

,review_creation_date,avg_score
0,2016-10-02,1.000000
1,2016-10-06,1.000000
2,2016-10-09,1.000000
3,2016-10-15,4.000000
4,2016-10-16,5.000000
...,...,...
631,2018-08-27,4.000000
632,2018-08-28,4.396018
633,2018-08-29,4.266667
634,2018-08-30,4.322667


In [12]:
fig2 = px.line(review_scores, x = 'review_creation_date', y = 'avg_score')
fig2.update_xaxes(title = 'дата', rangeslider_visible = True)
fig2.update_yaxes(title = 'средняя оценка')
fig2.show()

## Задание 3 (20 баллов)

Выберите топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [13]:
# Воспользуемся решением из второго задания

delivered_orders = orders[orders['order_status'] == 'delivered']['order_id']
filtered_items = items[['order_id', 'seller_id']]

sellers_to_delivered_orders = pd.merge(delivered_orders,
                                       filtered_items,
                                       left_on = 'order_id', right_on = 'order_id')

aggregated_sellers = sellers_to_delivered_orders.groupby('seller_id')\
                                                .agg(order_count = ('order_id', 'count'))\
                                                .reset_index()

active_sellers = aggregated_sellers[aggregated_sellers['order_count'] > 20]['seller_id']

filtered_sellers = sellers[['seller_id', 'seller_state']]
sellers_to_states = pd.merge(active_sellers, filtered_sellers,
                             left_on = 'seller_id', right_on = 'seller_id')

filtered_items = items[['seller_id', 'order_id']]

sellers_to_orders = pd.merge(sellers_to_states, filtered_items,
                             left_on = 'seller_id', right_on = 'seller_id')

filtered_orders = orders[orders['order_status'] == 'delivered'][['order_id', 'customer_id']]

sellers_to_customers = pd.merge(sellers_to_orders, filtered_orders,
                                left_on = 'order_id', right_on = 'order_id').drop('order_id', axis = 1)

filtered_customers = customers[['customer_id', 'customer_state']]

sellers_to_customer_states = pd.merge(sellers_to_customers, filtered_customers,
                                      left_on = 'customer_id', right_on = 'customer_id').drop('customer_id', axis = 1)

sellers_to_customer_states['is_other_state'] = sellers_to_customer_states[['seller_state', 'customer_state']]\
                                               .apply(lambda x: 1 if x['seller_state'] != x['customer_state']\
                                                                  else 0, \
                                                      axis = 1)

sellers_to_customer_states = sellers_to_customer_states.groupby('seller_id')\
                                                       .agg(count_total = ('seller_state', 'count'),
                                                            count_other_state = ('is_other_state', 'sum'))\
                                                       .reset_index()

sellers_to_customer_states['share'] = sellers_to_customer_states['count_other_state']/ \
                                      sellers_to_customer_states['count_total']

chosen_sellers = sellers_to_customer_states.sort_values(by = ['share', 'count_total'], ascending = False)[:5][['seller_id']]

In [14]:
chosen_sellers

,seller_id
164,2dee2ce60de9709b1a24083217181a1f
108,1ce3ae5a399804d1a87e706f8a813c3e
463,83deb69e889cf80f82be1dc6d5f2d486
822,f62d7bb4a613ec758a03342d46b580b2
445,7ecd59e5e20407131822c1a68ac59c1f


In [15]:
chosen_sellers_to_own_locations = pd.merge(chosen_sellers, sellers,
                                           left_on = 'seller_id', right_on = 'seller_id')

filtered_locations = locations.drop(['geolocation_city', 'geolocation_state'], axis = 1)

aggregated_locations = filtered_locations.groupby('geolocation_zip_code_prefix')\
                                         .agg(lat = ('geolocation_lat', 'mean'),
                                              lng = ('geolocation_lng', 'mean'))\
                                         .reset_index()

chosen_sellers_to_own_coords = pd.merge(chosen_sellers_to_own_locations, aggregated_locations,
                                        left_on = 'seller_zip_code_prefix', right_on = 'geolocation_zip_code_prefix')\
                                 .drop(['seller_zip_code_prefix', 'geolocation_zip_code_prefix'], axis = 1)\
                                 .sort_values(by = ['seller_id'])

chosen_sellers_to_own_coords

,seller_id,seller_city,seller_state,lat,lng
1,1ce3ae5a399804d1a87e706f8a813c3e,jaguaruna,SC,-28.633921,-49.024157
0,2dee2ce60de9709b1a24083217181a1f,sinop,MT,-11.834552,-55.508110
4,7ecd59e5e20407131822c1a68ac59c1f,brasilia,DF,-15.786284,-47.889118
2,83deb69e889cf80f82be1dc6d5f2d486,florianopolis,SC,-27.419842,-48.426466
3,f62d7bb4a613ec758a03342d46b580b2,vicosa,MG,-20.754735,-42.877520


In [16]:
filtered_items = items[['seller_id', 'order_id']]

chosen_sellers_to_items = pd.merge(chosen_sellers, filtered_items,
                                   left_on = 'seller_id', right_on = 'seller_id')

filtered_orders = orders[orders['order_status'] == 'delivered'][['order_id', 'customer_id']]

chosen_sellers_to_customers = pd.merge(chosen_sellers_to_items, filtered_orders,
                                       left_on = 'order_id', right_on = 'order_id').drop(['order_id'], axis = 1)


chosen_sellers_to_customer_locations = pd.merge(chosen_sellers_to_customers, customers,
                                                left_on = 'customer_id', right_on = 'customer_id')\
                                         .drop(['customer_id', 'customer_unique_id'], axis = 1)

chosen_sellers_to_customer_coords = pd.merge(chosen_sellers_to_customer_locations, aggregated_locations,
                                             left_on = 'customer_zip_code_prefix',
                                             right_on = 'geolocation_zip_code_prefix')\
                                      .drop(['customer_zip_code_prefix', 'geolocation_zip_code_prefix'], axis = 1)


chosen_sellers_to_customer_coords = \
                chosen_sellers_to_customer_coords.sort_values(by = ['seller_id'])\
                .groupby(['seller_id', 'customer_city', 'customer_state', 'lat', 'lng'])\
                .size().reset_index(name = 'customer_count')

In [17]:
chosen_sellers_to_customer_coords

,seller_id,customer_city,customer_state,lat,lng,customer_count
0,1ce3ae5a399804d1a87e706f8a813c3e,americana,SP,-22.753451,-47.327016,2
1,1ce3ae5a399804d1a87e706f8a813c3e,americana,SP,-22.743739,-47.336997,1
2,1ce3ae5a399804d1a87e706f8a813c3e,belem,PA,-1.433681,-48.469773,1
3,1ce3ae5a399804d1a87e706f8a813c3e,belo horizonte,MG,-19.916749,-43.901830,1
4,1ce3ae5a399804d1a87e706f8a813c3e,brasilia,DF,-15.835700,-47.835614,1
...,...,...,...,...,...,...
184,f62d7bb4a613ec758a03342d46b580b2,sao paulo,SP,-23.576192,-46.723989,1
185,f62d7bb4a613ec758a03342d46b580b2,sao paulo,SP,-23.571454,-46.752200,1
186,f62d7bb4a613ec758a03342d46b580b2,sao paulo,SP,-23.568755,-46.653409,1
187,f62d7bb4a613ec758a03342d46b580b2,sao paulo,SP,-23.560481,-46.682498,1


In [18]:
from plotly.validators.scatter.marker import SymbolValidator

raw_symbol = SymbolValidator().values[24]

fig = px.scatter_geo(chosen_sellers_to_customer_coords, lat = 'lat', lon = 'lng',
                     color = "seller_id",
                     size = 'customer_count',
                     hover_name = "customer_city",
                     hover_data = ["customer_state", "customer_count"],
                     projection = "natural earth",
                     color_discrete_sequence =  ['red', 'yellow', 'green', 'blue', 'purple'])


fig.add_trace(go.Scattergeo(lon = chosen_sellers_to_own_coords['lng'],
                            lat = chosen_sellers_to_own_coords['lat'],
                            marker_symbol = raw_symbol,
                            hovertext = 'seller_id = ' + chosen_sellers_to_own_coords['seller_id'],
                            name = 'seller locations',
                            marker=dict(color = ['red', 'yellow', 'green', 'blue', 'purple'],
                                        line = dict(color = 'Black', width = 3),
                                        size = 20)))

fig.show()

# Часть 2. Dash

В данной части нужно сделать и сдать один .py файл (можно назвать dash_task.py), который будет разворачивать страничку с нарисованными графиками (с помощью Dash)

## Задание 4 (20 баллов)

Сделайте сайт, где будет отображаться следующее:

* График, показывающий распределение по категориям продаж (то есть по тому, что продают селлеры) в штате

* График, показывающий распределение по категориям покупок (то есть по тому, что покупают пользователи) в штате

* Фильтр по штатам (должна быть возможность как выбрать отдельный штат, так и вся Бразилия в целом)

* Возможность выбрать, какие статусы продаж учитывать (чтобы можно было выбрать несколько статусов)

* Слайдер по датам (чтобы можно было выбрать даты, по которым можем смотреть результаты)


In [21]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 67.6 MB/s eta 0:00:00


In [28]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Анализ продаж и покупок"),
    dcc.Dropdown(
        id='state-dropdown',
        options=[
            {'label': 'Весь Бразилия', 'value': 'ALL'},
        ],
        value='ALL'
    ),
    dcc.Checklist(
        id='status-checklist',
        options=[
            {'label': 'Статус 1', 'value': 'STATUS1'},

        ],
        value=['@Anyazed STATUS1']
    ),
    dcc.RangeSlider(
        id='date-slider',
        min=0,
        max=10,
        step=1,
        value=[3, 7]
    ),
    dcc.Graph(id="sales-graph"),
    dcc.Graph(id="purchases-graph")
])

@app.callback(
    [Output("sales-graph", "figure"),
     Output("purchases-graph", "figure")],
    [Input("state-dropdown", "value"),
     Input("status-checklist", "value"),
     Input("date-slider", "value")]
)
def update_graphs(state, status, date_range):

    fig1 = px.bar(x=["a", "b", "c"], y=[1, 3, 2], title="Продажи")
    fig2 = px.bar(x=["a", "b", "c"], y=[2, 1, 3], title="Покупки")

    return fig1, fig2

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## Задание 5 (15 баллов)

Добавить карту, на котором можно будет посмотреть на количество продавцов или на количество покупателей (на него также должны работать фильтры по датам и по статусам, реализованные в прошлом занятии). Аггрегацию нужно делать по штатам. Чтобы вам помочь, вот snippet кода, который можно использовать для отображения:

In [30]:
import json
import pandas as pd
import plotly as plt
import plotly.express as px
from urllib.request import urlopen

# Brazil coordinates / shape
with urlopen(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
) as response:
    brazil = json.load(response)

# Since the database doesn't have an ID or feature using which values will be mapped between the coordinate/shape database and soybean database, we are adding an ID ourselves.
for feature in brazil["features"]:
    feature["id"] = feature["properties"]["sigla"]

# Здесь в качестве id, который будет отображаться, указано сокращение названия штата (как и в наших данных)

In [31]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


app = dash.Dash(__name__)


app.layout = html.Div([
    dcc.Dropdown(
        id='seller_or_customer',
        options=[
            {'label': 'Продавцы', 'value': 'sellers'},
            {'label': 'Покупатели', 'value': 'customers'}
        ],
        value='sellers'
    ),
    dcc.Graph(id='map')
])


@app.callback(
    Output('map', 'figure'),
    [Input('seller_or_customer', 'value')]
)
def update_map(seller_or_customer):
    if seller_or_customer == 'sellers':
        df = sellers.groupby('seller_state').size().reset_index(name='counts')
    else:
        df = customers.groupby('customer_state').size().reset_index(name='counts')

    fig = px.choropleth(df, geojson=brazil, locations='seller_state', color='counts',
                        color_continuous_scale="Viridis",
                        range_color=(0, 12),
                        labels={'counts':'количество'},
                        title="@Anyazed: Количество продавцов или покупателей по штатам"
                       )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

*Hint: обратите внимание, что покупатели и продавцы могут быть не во всех штатах, а эжто значит, что они могут не отображаться на карте. Эту ситуацию нужно обрабатывать и поправить*

## Задание 6 (15 баллов)

Сделайте карту кликабельной: по клику на штат должен показываться сам штат (и только он), все остальные графики обновляются по выбранному штату

In [32]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='brazil-map',
        figure=fig,
        config={'clickmode': 'event+select'}
    ),
    dcc.Graph(id='another-graph'),
    html.Div(id='output')
])

@app.callback(
    Output('output', 'children'),
    [Input('brazil-map', 'clickData')]
)
def display_click_data(clickData):
    if clickData is not None:
        state_id = clickData['points'][0]['location']
        return 'Вы кликнули по штату: {}'.format(state_id)

@app.callback(
    Output('another-graph', 'figure'),
    [Input('brazil-map', 'clickData')]
)
def update_graph(clickData):
    if clickData is not None:
        state_id = clickData['points'][0]['location']
        # Здесь вы можете создать новый график на основе выбранного штата
        fig = go.Figure()
        return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>